In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Customer-Churn'

In [2]:
from dataclasses import dataclass
from pathlib import Path

# enitity
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    encoder_name: str
    test_size: float
    all_cols: list
    target_col: str
    features: dict

In [3]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    """
    A class to manage configuration settings for this data science project.
    """

    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
        schema_filepath: str = SCHEMA_FILE_PATH
    ) -> None:
        """
        Initializes the ConfigurationManager with default or provided file paths.

        Args:
        - config_filepath (str, optional): Path to the main configuration file.
        - params_filepath (str, optional): Path to the parameters file.
        - schema_filepath (str, optional): Path to the schema file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        Retrieves the data transformation configuration.

        Returns:
        - DataTransformationConfig: Configuration settings for data transformation.
        """
        config = self.config.data_transformation
        all_cols = list(self.schema.COLUMNS.keys())
        target_col = self.schema.TARGET_COLUMN
        features = self.schema.FEATURES

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            encoder_name=config.encoder_name,
            test_size=config.test_size,
            all_cols=all_cols,
            target_col=target_col.name,
            features=features,
        )

        return data_transformation_config

In [5]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from CustomerChurn.utils.common import save_bin, save_json
from pathlib import Path
from sklearn.model_selection import train_test_split  
from CustomerChurn import logger
import os
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [6]:
class DataTransformation:
    """
    A class for performing data transformation based on provided configuration.
    """

    def __init__(self, config: DataTransformationConfig) -> None:
        """
        Initializes the DataTransformation instance with the provided configuration.

        Args:
        - config (DataTransformationConfig): Configuration settings for data transformation.
        """
        self.config = config

    def transform_data(self) -> None:
        """
        Reads data, performs transformations, and saves the preprocessed data.

        Returns:
        - None
        """
        try:
            data = pd.read_csv(self.config.data_path)
            data.drop(["customerID"], axis=1, inplace=True)

            # Convert total charges to numeric
            data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

            categorical_features = self.config.features.categorical
            numeric_features = self.config.features.numerical

            logger.info(f"Numeric features: {numeric_features}")
            logger.info(f"Categorical features: {categorical_features}")

            numeric_transformer = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", MinMaxScaler(feature_range=(0, 1)))
                ]
            )

            categorical_transformer = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("encoder", OrdinalEncoder()),
                    ("normalizer", MinMaxScaler())
                ]
            )

            preprocessor = ColumnTransformer(
                transformers=[
                    ("num", numeric_transformer, numeric_features),
                    ("cat", categorical_transformer, categorical_features),
                ]
            )

            preprocessed_data = preprocessor.fit_transform(data)
            preprocessed_df = pd.DataFrame(preprocessed_data, columns=data.columns)

            save_bin(data=preprocessor, path=Path(os.path.join(self.config.root_dir, self.config.encoder_name)))
            preprocessed_df.to_csv(os.path.join(self.config.root_dir, "encoded_data.csv"), index=False)
            logger.info(f"The data shape - {preprocessed_data.shape}")

        except Exception as e:
            raise e

    def split_data(self) -> None:
        """
        Reads encoded data, splits it into training and test sets, and saves the sets.

        Returns:
        - None
        """
        try:
            data = pd.read_csv(os.path.join(self.config.root_dir, "encoded_data.csv"))

            X = data.drop(self.config.target_col, axis=1)
            y = data[self.config.target_col]

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

            train = pd.concat([X_train, y_train], axis=1)
            test = pd.concat([X_test, y_test], axis=1)

            train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
            test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

            logger.info("Split data into training and test sets")
            logger.info(f"Train data shape - {train.shape}")
            logger.info(f"Test data shape - {test.shape}")

        except Exception as e:
            raise e


In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()
    data_transformation.split_data()
except Exception as e:
    raise e

[2024-02-01 16:26:40,383: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-01 16:26:40,386: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 16:26:40,390: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-01 16:26:40,391: INFO: common: created directory at: artifacts]
[2024-02-01 16:26:40,392: INFO: common: created directory at: artifacts/data_transformation]
[2024-02-01 16:26:40,415: INFO: 4043610263: Numeric features: ['TotalCharges', 'tenure', 'MonthlyCharges']]
[2024-02-01 16:26:40,416: INFO: 4043610263: Categorical features: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'SeniorCitizen', 'Churn']]
[2024-02-01 16:26:40,450: INFO: common: binary file saved at: artifacts/data_transformation/encoder.joblib]
[2024-02-01 16:26:40,506: 